Citation: Richardson G, Knudby A, Chen W, Sawada M, Lovitt J, He L, et al. (2023) Dense neural network outperforms other machine learning models for scaling-up lichen cover maps in Eastern Canada. PLoS ONE 18(11): e0292839. https://doi.org/10.1371/journal.pone.0292839

In [1]:
import os, shutil, time, csv,math,scipy,joblib,matplotlib
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pearsonr,gaussian_kde
from sklearn.inspection import permutation_importance
#displaying data
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow,figure
#TF imports
import tensorflow as tf

### Metrics functions

In [2]:
#These are general metrics functions
def dtime():
    return (datetime.now().strftime("%m%d%Y_%H%M%S"))
def calculate_metrics(results):
    '''Calculated MAE,r2,pearsons from a results pd.dataframe'''
    MAElist= []      #make a list of absolute errors
    for i in range(len(results)):
        MAElist.append((abs(results[0][i]*100-results["pred"][i]*100)))
    mean_MAE=np.round(np.mean(MAElist),3) #calculate mean
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(results[0], results['pred'])
    r_squared=r_value**2
    corr, _ = pearsonr(results["pred"], results[0])
    return mean_MAE,round(r_squared,2),round(corr,3)
def time_and_metrics(Results):
    dt_string=dtime()
    mae,r2,person=calculate_metrics(Results)#calculates metrics
    return mae,r2,dt_string

### Plotting Functions

In [1]:
#These functions are for plotting data.
def make_scatter_from_results(Results,Results_str,modelplotstr,path):
    import matplotlib.ticker as mtick
    '''Creates pretty scatterplot and saves it in a meaningful way.
    This plot should be used for all Lichen % comparisons between true
    and predicted.''' 
    '''Consider removing empty space on right of figure'''
    mae,r2,dt_string=time_and_metrics(Results)
    output_path=path+'\\'+modelplotstr
    image_path='{}\\{}_{}-{}_{}.png'.format(output_path,Results_str,mae,r2,dt_string)
    if Path(output_path).exists()==False:
        os.makedirs(output_path)
    x,y=Results[0],Results["pred"]
    #work on frequency colouring
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy/2) 
    fig = plt.figure(figsize=(7.75,7))#define plot size, 7,6 makes SP a square
    #seperate space for plot and Cbar
    gs=matplotlib.gridspec.GridSpec(1,2, width_ratios=[6,.2])
    ax1 = plt.subplot(gs[0])
    plt.xlabel("True Values",size=18)
    plt.ylabel("Predicted",size=18)
    plt.xticks(np.arange(0,1,step=.1),fontsize=14)
    plt.yticks(np.arange(0,1,step=.1),fontsize=14)
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    ax2 = plt.subplot(gs[1])
    #plot scatter, 1:1 line
    sc=ax1.scatter(x, y, c=z,cmap='viridis')
    #used to create 1:1 line
    line=ax1.plot(Results[0], Results[0], color = 'black', label = '1:1 Line',alpha=0.6)
    #used to create regression line
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(Results[0], Results['pred'])
    x_vals = np.array((0,ax1.get_xlim()[1]))
    y_vals = intercept + slope * x_vals
    reg=ax1.plot(x_vals,y_vals,color = 'red',label = 'Linear Regression',alpha=0.6)
    #add # samples to items as smalle r
    ax1.legend(loc='upper left',fontsize=16,frameon=False)
    #Title of plot
    ax1.set_title('MAE = {}         R\u00b2 = {}         n = {}'.format(np.round(mae*.01,4)
                                                                          ,r2,str(len(Results[0]))),fontsize=18,pad=10)
    cbar=plt.colorbar(sc, cax=ax2)
    cbar.remove() 
    #plt.tight_layout()
    plt.savefig(image_path,dpi=500) #save image

def plot_hist_save(model,temp_weights,val_dataset,model_history,path,NN_str):
    '''Saves the model history in a meaningful way. This function should
    be used for every NN model which is tested, and for Hyperparam tuning.'''
    mae=get_model_mae(model,temp_weights,val_dataset)
    dt_string = dtime()
    fig = plt.figure(figsize=(6,4))
    plt.xlabel('Epoch',size=10)
    plt.ylabel('MAE Loss',size=10)
    plt.title('Model Loss per Epoch',size=12)
    plt.plot(model_history.history['loss'],'cornflowerblue')
    plt.plot(model_history.history['val_loss'],'darkred')
    plt.legend(['Training Loss','Validation Loss'],loc='upper right',fontsize=9,frameon=True)
    plt.grid()
    plt.savefig('{}\{}\Histplot-{}_{}.png'.format(path,NN_str,str(round(mae,5)),dt_string),dpi=400)#save image
def Feature_importance(rf_reg,test,test_labels,imgpath):
    result = permutation_importance(
        rf_reg, test, test_labels, n_repeats=10, random_state=42, n_jobs=2)

    forest_importances = pd.Series(result.importances_mean)
    x=["B2","B3","B4","B5","B6","B7","B8","B11","B12"]
    fig, ax = plt.subplots()
    forest_importances.plot.bar(ax=ax, color='black')
    ax.set_title("Feature importances using permutation on RF model",fontsize=12)
    ax.set_xticks(range(9),labels=x,rotation=0)
    ax.set_ylabel("Mean accuracy decrease",fontsize=12),ax.set_xlabel("Sentinel-2 Bands",fontsize=12)
    #fig.tight_layout()
    plt.savefig(imgpath,dpi=400)
    plt.show()

### NN Related Functions

In [4]:
#These functions are made for Dense NN's
def get_model_mae(model,temp_weights,val_dataset):
    model.load_weights(temp_weights)
    _,mae,_ = model.evaluate(val_dataset)
    return mae
def save_nn_for_log(model,temp_weights,val_dataset,path,NN_str):
    '''saves the model weights in a meaningful way'''
    dt_string = dtime()
    mae=get_model_mae(model,temp_weights,val_dataset)
    if Path('{}\{}'.format(path,NN_str)).exists()==False:
        os.makedirs('{}\{}'.format(path,NN_str))
    savepath=r'{}\{}\Weights-{}_{}.h5'.format(path,NN_str,str(round(mae,5)),dt_string)
    model.save(savepath)
    print('Weights saved')
def nnDset_to_Results(model,nnDataset):
    '''evaluates TF dataset and creates Results pd.dataframe'''
    results_list=[]
    for element in nnDataset.as_numpy_iterator(): 
        result=model.predict(element[0])
        results_list.append([element[1]*100,result*100])
    x,y=[],[]
    for i in results_list[0][0]:
        x.append(abs(i)/100)
    for i in results_list[0][1]:
        y.append(abs(i[0])/100)
    results_nn=pd.DataFrame(list(zip(x,y)),columns = [0,'pred'])
    return results_nn